In [6]:
# validate env setup
import os
from pyspark.sql import SparkSession

# ensure that these variables are set in your env, either manually or through your shell config file or venv activation script
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "/Users/lydialim/pyspark-eda-demo/.venv/lib/python3.10/site-packages/pyspark"

# initialize spark session
spark = SparkSession.builder.getOrCreate()

# additional spark session configurations
spark.conf.set("spark.sql.session.timeZone", "Asia/Singapore")  # UTC+8
spark.sparkContext.setLogLevel("ERROR")  # Options: ALL, DEBUG, ERROR, FATAL, INFO, OFF, TRACE, WARN
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")


In [7]:
# import libs
from pyspark.sql.functions import (
    aes_encrypt,
    base64,
    col,
    length,
    lit,
    when,
    from_unixtime,
    from_utc_timestamp,
    to_timestamp,
    date_format,
    regexp_replace,
    split,
    expr,
    element_at,
    year
)
from pyspark.sql import DataFrame
from sparkdq.checks import NullCheckConfig, TimestampBetweenCheckConfig
from sparkdq.engine import BatchDQEngine
from sparkdq.management import CheckSet
from sparkdq.core import Severity


In [8]:
df = spark.read.parquet("data_fixtures/cc_sample_transaction.parquet")
df.printSchema()
df.show(5)

root
 |-- trans_date_trans_time: string (nullable = true)
 |-- cc_num: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: string (nullable = true)
 |-- person_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- merch_lat: string (nullable = true)
 |-- merch_long: string (nullable = true)
 |-- is_fraud: string (nullable = true)
 |-- merch_zipcode: string (nullable = true)
 |-- merch_last_update_time: string (nullable = true)
 |-- merch_eff_time: string (nullable = true)
 |-- cc_bic: string (nullable = true)

+---------------------+-----------

In [9]:
# define some functions that will help us clean the data
def to_utc8_hm(c):
    """
    Accepts Column c whose values may be:
      • purely numeric, ≥15 digits → µs since epoch
      • purely numeric, 12–14 digits → ms since epoch
      • anything else → literal 'yyyy-MM-dd HH:mm:ss'
    Returns a Column of timestamps in UTC+8
    """
    c_str = c.cast("string")
    is_digits = c_str.rlike("^[0-9]+$")

    # 1) parse into a UTC Timestamp
    ts_utc = (
        when(
            is_digits & (length(c_str) >= 15), from_unixtime(c_str.cast("double") / 1e6)
        )  # µs → seconds
        .when(
            is_digits & (length(c_str).between(12, 14)),
            from_unixtime(c_str.cast("double") / 1e3),
        )  # ms → seconds
        .otherwise(to_timestamp(c_str, "yyyy-MM-dd HH:mm:ss"))  # literal parse
    )

    # 2) shift to UTC+8
    ts_utc8 = from_utc_timestamp(ts_utc, "GMT+8")

    return ts_utc8

def format_timestamp(ts_col):
    """
    Accepts a timestamp string and returns a formatted timestamp string.
    Output format: 'yyyy-MM-dd HH:mm:ss.SSSSSS +08:00'
    """
    return date_format(ts_col, "yyyy-MM-dd HH:mm:ss.SSSSSS +08:00")

def clean_and_split_names(
    df: DataFrame,
    input_col: str = "person_name",
    first_col: str = "first",
    last_col: str = "last",
) -> DataFrame:
    """
    Given a DataFrame with `input_col` containing raw names,
    returns a new DataFrame with `first_col` and `last_col`.

    Assumptions:
    - Names are separated by a space
    - Names are always two words
    - Names are always in the format "First Last"
    - Names are always alphabetic
    - Names are never empty
    """
    return (
        df
        # 1) Turn anything that isn't A–Z/a–z into a space
        .withColumn("_cleaned", regexp_replace(col(input_col), "[^A-Za-z]", " "))
        # 2) Split into tokens on spaces
        .withColumn("_tokens", split(col("_cleaned"), " "))
        # 3) Drop empty tokens
        .withColumn("_tokens", expr("filter(_tokens, x -> x <> '')"))
        # 4) Extract first & second tokens
        .withColumn(first_col, element_at(col("_tokens"), 1)).withColumn(
            last_col, element_at(col("_tokens"), 2)
        )
        # 5) Drop the helpers
        .drop("_cleaned", "_tokens")
    )

First, we deal with the timestamp columns by standardizing the format.

In [10]:
timestamp_cols = ["merch_eff_time", "merch_last_update_time", "trans_date_trans_time"]

# overwrites the existing timestamp columns inplace
for ts in timestamp_cols:
    df = df.withColumn(ts, to_utc8_hm(col(ts)))

df.show(5, truncate=False)

+---------------------+----------------+----------------------------------+-------------+------+--------------------+------+----------------------------+--------------+-----+-----+-------+---------+--------+---------------------------------+----------+--------------------------------+------------------+-----------+--------+-------------+----------------------+-------------------+-----------+
|trans_date_trans_time|cc_num          |merchant                          |category     |amt   |person_name         |gender|street                      |city          |state|zip  |lat    |long     |city_pop|job                              |dob       |trans_num                       |merch_lat         |merch_long |is_fraud|merch_zipcode|merch_last_update_time|merch_eff_time     |cc_bic     |
+---------------------+----------------+----------------------------------+-------------+------+--------------------+------+----------------------------+--------------+-----+-----+-------+---------+--------+---

# Identifying Dirty Data
- Everything starts from common sense and business logic. 
- Discoverability and documentation is also important. Without anyone actually using the data for anything, it's usually hard to detect systemic issues with the data.

Data Validation
- Catch dirty data in the records. In reality you wouldn't actually remove those records, but add more ways to handle them as you get alerted to their existence from schema/rule violations. 
- A downside of quietly handling things is that over time this might build up unneeded complexity in the pipelines. It shouldn't all fall on engineering to solve, though. I am a proponent of letting things burn once in awhile if that's what it takes to bring positive changes. 

- For python, schema validation through Pydantic is a good start. If using databricks, can consider this new framework: https://databrickslabs.github.io/dqx/docs/motivation/
- For this assignment I had the opportunity to try a very new library called `sparkdq`. It works like Great Expectations, but for Spark.

Data Observability
- Catch unusual trends. Records that conform to the validation rules might not mean that all is well - what if the null values in a nullable column are suddenly greater % than usual? For paid solutions with a nice UI, tools like Monte Carlo and Data Dog can help.
- Data engineering can champion the use of data observability tools to catch unusual trends, and empower domain teams to take ownership of their data.

Integration Patterns
We can either:
1. fail-fast (throw error and fail the entire pipeline) or 
2. quarantine the records (log the error / dirty data for later analysis, while the rest of the pipeline continues).

In this case, we will quarantine the records. (I am making the assumption that the pipeline is run in a batch fashion, and the dirty data is not critical to the pipeline or the downstream processes.)


In [11]:
check_set = CheckSet()
null_check = NullCheckConfig(check_id="my-null-check", columns=["cc_num", 'trans_date_trans_time', 'trans_num'])
# example of timestamp check
timestamp_check = TimestampBetweenCheckConfig(
    check_id="allowed_record_date_range",
    columns=["trans_date_trans_time", "merch_eff_time", "merch_last_update_time"],
    min_value="2000-01-01 00:00:00",
    max_value="2025-12-31 23:59:59",
    inclusive=(True, True),
    severity=Severity.CRITICAL
)

check_set.add_check(null_check)
check_set.add_check(timestamp_check)

result = BatchDQEngine(check_set).run_batch(df)
print(result.summary())

# save the failed records to a quarantine folder for later analysis
result.fail_df().write.format("json").save("quarantine", mode="overwrite")


Validation Summary (2025-05-27 13:00:04)
Total records:   1,296,675
Passed records:  1,051,939
Failed records:  244,736
Warnings:        0
Pass rate:       81.00%


In [12]:
# remove the failed records from the original dataframe
fail_df_subset = result.fail_df().select(*result.fail_df().columns[:-3])
df = df.subtract(fail_df_subset)
print("Remaining records after removing failed records:")
df.count()


Remaining records after removing failed records:


1063595

In [13]:
# format the timestamp columns

for ts in timestamp_cols:
    df = df.withColumn(ts, format_timestamp(col(ts)))

df.show(5, truncate=False)


+---------------------------------+----------------+-------------------------------------+------------+------+-------------------------+------+-------------------------------+----------------+-----+-----+-------+--------+--------+----------------------------------+----------+--------------------------------+---------+------------------+--------+-------------+---------------------------------+---------------------------------+-----------+
|trans_date_trans_time            |cc_num          |merchant                             |category    |amt   |person_name              |gender|street                         |city            |state|zip  |lat    |long    |city_pop|job                               |dob       |trans_num                       |merch_lat|merch_long        |is_fraud|merch_zipcode|merch_last_update_time           |merch_eff_time                   |cc_bic     |
+---------------------------------+----------------+-------------------------------------+------------+------+------

Then, we split the person_name columns into first and last name.

In [14]:
df = clean_and_split_names(df, "person_name", "first", "last")
df = df.drop("person_name")
df.show(5, truncate=False)

+---------------------------------+----------------+-------------------------------------+------------+------+------+-------------------------------+----------------+-----+-----+-------+--------+--------+----------------------------------+----------+--------------------------------+---------+------------------+--------+-------------+---------------------------------+---------------------------------+-----------+-----+----------+
|trans_date_trans_time            |cc_num          |merchant                             |category    |amt   |gender|street                         |city            |state|zip  |lat    |long    |city_pop|job                               |dob       |trans_num                       |merch_lat|merch_long        |is_fraud|merch_zipcode|merch_last_update_time           |merch_eff_time                   |cc_bic     |first|last      |
+---------------------------------+----------------+-------------------------------------+------------+------+------+-----------------

# PII Sanitization
Sanitizing PII is a tricky topic and depends on the downstream use cases and regulations surrounding the data.

# Considerations for data storage
- For the purposes of this assignment, we have the CC owner information (gender, job, dob, etc) and the Merchant details stored alongside the transaction data. 
- But ideally these details should be normalized and stored in a separate dimension table, with a foreign key to the transaction table. Then access to the database containing the details should be RBAC-based and controlled through policies.
- When storing the PII such as first and last name, we should store the hashed or encrypted version of the name. More on that below.
- The key to the encryption should be tightly controlled and exposed only to certain roles.


# Considerations for masking and hashing
If we choose to entirely mask or hash the PII, then downstream analysis might be hamstrung. 

For example, if we mask the credit card number, we would not be able to use it for any analysis / joining to BIN number lists to identify the issuing bank / card type. For example an e-commerce site may want to use the first 6 to 8 digits of the credit card number to identify the issuing bank / card type to offer rewards to the cardholder, upon eligible transactions.


Options for masking and hashing:
- Partial anonymization: J*** D**, easy to implement but weak and identity thieves can still infer an identity using a few more personal details.
- Full masking: XXXX XXX, strong and simple to implement, but cannot be used for any analysis. 
- Hashing (unsalted): Pros: Same input ⇒ same hash ⇒ you can still group or join records by name. Cons: Can be reverse-engineered / vulnerable to rainbow table (pre-computed key-value pairs) attacks if leaked.
- Hashing (salted): Prevents rainbow table attacks since knowing the hash alone is not enough to reverse-engineer the original input. 

# Considerations for encryption
More flexible and secure than hashing, but requires more resources to encrypt and decrypt data. 
- This preserves the ability to use the PII for unique-user analysis because the same input value will always produce the same output. 
- In addition, teams that really need to access the raw PII data can be granted access to the encryption key.

Ref: 
- https://www.chargeflow.io/blog/bank-identification-numbers-bin-the-backbone-of-payment-processing
- https://www.pdpc.gov.sg/-/media/files/pdpc/pdf-files/advisory-guidelines/guide-to-basic-anonymisation-(updated-24-july-2024).pdf


In [15]:
def mask_credit_card(df: DataFrame, input_col: str, output_col: str) -> DataFrame:
    """
    Always masks at least one digit in the middle, even if the total length is <= 12.
    """
    df = df.withColumn(input_col, col(input_col).cast("string"))
    masked = expr(
        f"""CASE
             WHEN length({input_col}) > 12 THEN
               concat(
                 substr({input_col}, 1, 8),
                 repeat('*', length({input_col}) - 12),
                 substr({input_col}, -4, 4)
               )
             ELSE
               concat(
                 substr({input_col}, 1, 8),
                 '*',
                 substr({input_col}, 9, length({input_col})-8)
               )
           END"""
    )
    return df.withColumn(output_col, masked)

def encrypt_column(
    df: DataFrame,
    input_col: str,
    output_col: str,
    key: bytes
) -> DataFrame:
    """
    Encrypt `input_col` → `output_col` using Spark's native AES_ENCRYPT (ECB mode).

    :param df:         input DataFrame
    :param input_col:  name of plaintext (string) column
    :param output_col: name of resulting ciphertext (binary) column
    :param key:        raw AES key (must be 16, 24, or 32 bytes)
    """
    # wrap the key as a literal binary
    key_lit = lit(key)
    return df.withColumn(output_col, base64(aes_encrypt(input=col(input_col), key=key_lit)))

In [16]:
# truncate dob to year - this is a way to "generalize" the dob
df = df.withColumn("dob", year(col("dob")))
df.show(5, truncate=False)

+---------------------------------+----------------+-------------------------------------+------------+------+------+-------------------------------+----------------+-----+-----+-------+--------+--------+----------------------------------+----+--------------------------------+---------+------------------+--------+-------------+---------------------------------+---------------------------------+-----------+-----+----------+
|trans_date_trans_time            |cc_num          |merchant                             |category    |amt   |gender|street                         |city            |state|zip  |lat    |long    |city_pop|job                               |dob |trans_num                       |merch_lat|merch_long        |is_fraud|merch_zipcode|merch_last_update_time           |merch_eff_time                   |cc_bic     |first|last      |
+---------------------------------+----------------+-------------------------------------+------------+------+------+-----------------------------

In [17]:
df = mask_credit_card(df, "cc_num", "cc_num")
df.show(5, truncate=False)

+---------------------------------+----------------+-------------------------------------+------------+------+------+-------------------------------+----------------+-----+-----+-------+--------+--------+----------------------------------+----+--------------------------------+---------+------------------+--------+-------------+---------------------------------+---------------------------------+-----------+-----+----------+
|trans_date_trans_time            |cc_num          |merchant                             |category    |amt   |gender|street                         |city            |state|zip  |lat    |long    |city_pop|job                               |dob |trans_num                       |merch_lat|merch_long        |is_fraud|merch_zipcode|merch_last_update_time           |merch_eff_time                   |cc_bic     |first|last      |
+---------------------------------+----------------+-------------------------------------+------------+------+------+-----------------------------

In [18]:
# IRL this should be retrieved from a secure key vault or KMS
aes_key = b"0123456789ABCDEF"

In [19]:
df = encrypt_column(df, input_col="first", output_col="first", key=aes_key)
df = encrypt_column(df, input_col="last", output_col="last", key=aes_key)
df.show(5, truncate=False)

+---------------------------------+----------------+-------------------------------------+------------+------+------+-------------------------------+----------------+-----+-----+-------+--------+--------+----------------------------------+----+--------------------------------+---------+------------------+--------+-------------+---------------------------------+---------------------------------+-----------+--------------------------------------------+----------------------------------------------------+
|trans_date_trans_time            |cc_num          |merchant                             |category    |amt   |gender|street                         |city            |state|zip  |lat    |long    |city_pop|job                               |dob |trans_num                       |merch_lat|merch_long        |is_fraud|merch_zipcode|merch_last_update_time           |merch_eff_time                   |cc_bic     |first                                       |last                                  

In [20]:
# replace digits in street address with X to obscure the address
df = df.withColumn(
    "street",
    regexp_replace(col("street"), r"\d", "X")
)
df.show(5, truncate=False)

+---------------------------------+----------------+-------------------------------------+------------+------+------+-------------------------------+----------------+-----+-----+-------+--------+--------+----------------------------------+----+--------------------------------+---------+------------------+--------+-------------+---------------------------------+---------------------------------+-----------+--------------------------------------------+----------------------------------------------------+
|trans_date_trans_time            |cc_num          |merchant                             |category    |amt   |gender|street                         |city            |state|zip  |lat    |long    |city_pop|job                               |dob |trans_num                       |merch_lat|merch_long        |is_fraud|merch_zipcode|merch_last_update_time           |merch_eff_time                   |cc_bic     |first                                       |last                                  

In [21]:
# Check and see if there is a pattern to the job titles
unique_values = df.select("job").distinct().collect()
for row in unique_values[:10]:
    print(row.job)

Retail merchandiser
Librarian, academic
Designer, ceramics/pottery
Catering manager
Engineer, aeronautical
English as a second language teacher
Early years teacher
Occupational hygienist
Primary school teacher
Control and instrumentation engineer


One can argue that there is a pattern to the job titles, such that we can generalize them by using regex and other string manipulation functions. The downside is that these patterns are not always consistent and may change over time, rendering our logic obsolete.

But a better and more nuanced approach to generalize the job titles would be to:
1. Convert a fixed list of industries (that we would like to analyze by) into embeddings.
2. Convert the job titles into embeddings. Then use RAG to map each individual's job title to an industry. 
3. In the final dataset, we can then expose only the industry, but not the job title. 

The challenge here would be how to scalably run the embeddings and retrieval for each new user entity. 

In [22]:
# Convert the spark df into a pandas df for visualization
# df_pandas = df.toPandas()